# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
F. Walter  ->  F. Walter  |  ['F. Walter']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
A. Gould  ->  A. Gould  |  ['A. Gould']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
Arxiv has 42 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2404.12432


extracting tarball to tmp_2404.12432...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2404.12432/sample631.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'Table_filler.tex' from 'tmp_2404.12432/Table_filler.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'table_quiescent.tex' from 'tmp_2404.12432/table_quiescent.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.19/x64/lib/pytho

...

 done.
Retrieving document from  https://arxiv.org/e-print/2404.13031


extracting tarball to tmp_2404.13031... done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2404.13031/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'affi.tex' from 'tmp_2404.13031/affi.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'author' from 'tmp_2404.13031/author.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 76 bibliographic references in tmp_2404.13031/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2404.13047


extracting tarball to tmp_2404.13047... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.12432-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.12432) | **The JWST-SUSPENSE Ultradeep Spectroscopic Program: Survey Overview and  Star-Formation Histories of Quiescent Galaxies at 1 < z < 3**  |
|| M. Slob, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-04-22*|
|*Comments*| *Submitted to ApJ; 24 pages, 13 figures, 2 tables (excluding appendices)*|
|**Abstract**| We present an overview and first results from the Spectroscopic Ultradeep Survey Probing Extragalactic Near-infrared Stellar Emission (SUSPENSE), executed with NIRSpec on JWST. The primary goal of the SUSPENSE program is to characterize the stellar, chemical, and kinematic properties of massive quiescent galaxies at cosmic noon. In a single deep NIRSpec/MSA configuration, we target 20 distant quiescent galaxy candidates ($z=1-3$, $H_{AB}<23$), as well as 53 star-forming galaxies at $z=1-4$. With 16hr of integration and the G140M-F100LP dispersion-filter combination, we observe numerous Balmer and metal absorption lines for all quiescent candidates. We derive stellar masses (log$M_*/M_{\odot}\sim10.3-11.5$) and detailed star-formation histories (SFHs) and show that all 20 candidate quiescent galaxies indeed have quenched stellar populations. These galaxies show a variety of mass-weighted ages ($0.8-3.0$Gyr) and star formation timescales ($\sim0.5-4$Gyr), and four out of 20 galaxies were already quiescent by $z=3$. On average, the $z>1.75$ $[z<1.75]$ galaxies formed 50% of their stellar mass before $z=4$ $[z=3]$. Furthermore, the typical SFHs of galaxies in these two redshift bins ($z_{\text{mean}}=2.2$ and $z_{\text{mean}}=1.3$) indicate that galaxies at higher redshift formed earlier and over shorter star-formation timescales compared to lower redshifts. Although this evolution is naturally explained by the growth of the quiescent galaxy population over cosmic time, we cannot rule out that mergers and late-time star formation also contribute to the evolution. In future work, we will further unravel the early formation, quenching, and late-time evolution of these galaxies by extending this work with studies on their chemical abundances, |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.12443-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.12443) | **Multi-phase black-hole feedback and a bright [CII] halo in a Lo-BAL  quasar at $z\sim6.6$**  |
|| M. Bischetti, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2024-04-22*|
|*Comments*| *Submitted to ApJ*|
|**Abstract**| Although the mass growth of supermassive black holes during the Epoch of Reionisation is expected to play a role in shaping the concurrent growth of their host-galaxies, observational evidence of feedback at z$\gtrsim$6 is still sparse. We perform the first multi-scale and multi-phase characterisation of black-hole driven outflows in the $z\sim6.6$ quasar J0923+0402 and assess how these winds impact the cold gas reservoir. We employ the SimBAL spectral synthesis to fit broad absorption line (BAL) features and find a powerful ionized outflow on $\lesssim210$ pc scale, with a kinetic power $\sim2-100$\% of the quasar luminosity. ALMA observations of [CII] emission allow us to study the morphology and kinematics of the cold gas. We detect high-velocity [CII] emission, likely associated with a cold neutral outflow at $\sim0.5-2$ kpc scale in the host-galaxy, and a bright extended [CII] halo with a size of $\sim15$ kpc. For the first time at such an early epoch, we accurately constrain the outflow energetics in both the ionized and the atomic neutral gas phases. We find such energetics to be consistent with expectations for an efficient feedback mechanism, and both ejective and preventative feedback modes are likely at play. The scales and energetics of the ionized and atomic outflows suggest that they might be associated with different quasar accretion episodes. The results of this work indicate that strong black hole feedback is occurring in quasars at $z\gtrsim6$ and is likely responsible for shaping the properties of the cold gas reservoir up to circum-galactic scales. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.12585-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.12585) | **IGM damping wing constraints on the tail end of reionisation from the  enlarged XQR-30 sample**  |
|| B. Greig, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2024-04-22*|
|*Comments*| *21 pages, 9 figures and 2 tables. Accepted for publication in MNRAS*|
|**Abstract**| The attenuation of Ly$\alpha$ photons by neutral hydrogen in the intergalactic medium (IGM) at $z\gtrsim5$ continues to be a powerful probe for studying the epoch of reionisation. Given a framework to estimate the intrinsic (true) Ly$\alpha$ emission of high-$z$ sources, one can infer the ionisation state of the IGM during reionisation. In this work, we use the enlarged XQR-30 sample of 42 high-resolution and high-SNR QSO spectra between $5.8\lesssim\,z\lesssim\,6.6$ obtained with VLT/X-Shooter to place constraints on the IGM neutral fraction. This is achieved using our existing Bayesian QSO reconstruction framework which accounts for uncertainties such as the: (i) posterior distribution of predicted intrinsic Ly$\alpha$ emission profiles (obtained via covariance matrix reconstruction of the Ly$\alpha$ and N V emission lines from unattenuated high-ionisation emission line profiles; C IV, Si IV + O IV] and C III]) and (ii) distribution of ionised regions within the IGM using synthetic damping wing profiles drawn from a $1.6^3$ Gpc$^3$ reionisation simulation. Following careful quality control, we used 23 of the 42 available QSOs to obtain constraints/limits on the IGM neutral fraction during the tail-end of reionisation. Our median and 68th percentile constraints on the IGM neutral fraction are: $0.20\substack{+0.14\\-0.12}$ and $0.29\substack{+0.14\\-0.13}$ at $z = 6.15$~and 6.35. Further, we also report 68th percentile upper-limits of $\bar{x}_{\mathrm{H\,{\scriptscriptstyle I}}} < 0.21$, 0.20, 0.21 and 0.18 at $z = 5.8, 5.95, 6.05$~and 6.55. These results imply reionisation is still ongoing at $5.8\lesssim\,z\lesssim\,6.55$, consistent with previous results from XQR-30 (dark fraction and Ly$\alpha$ forest) along with other observational probes considered in the literature. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.13031-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.13031) | **OGLE-2015-BLG-0845L: A low-mass M dwarf from the microlensing parallax  and xallarap effects**  |
|| Z. Hu, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2024-04-22*|
|*Comments*| *13 pages, 10 figures. Submitted to Monthly Notices of the Royal Astronomical Society*|
|**Abstract**| We present the analysis of the microlensing event OGLE-2015-BLG-0845, which was affected by both the microlensing parallax and xallarap effects. The former was detected via the simultaneous observations from the ground and Spitzer, and the latter was caused by the orbital motion of the source star in a relatively close binary. The combination of these two effects led to a direct mass measurement of the lens object, revealing a low-mass ($0.14 \pm 0.05 M_{\odot}$) M-dwarf at the bulge distance ($7.6 \pm 1.0$ kpc). The source binary consists of a late F-type subgiant and a K-type dwarf of $\sim1.2 M_{\odot}$ and $\sim 0.9 M_{\odot}$, respectively, and the orbital period is $70 \pm 10$ days. OGLE-2015-BLG-0845 is the first single-lens event in which the lens mass is measured via the binarity of the source. Given the abundance of binary systems as potential microlensing sources, the xallarap effect may not be a rare phenomenon. Our work thus highlights the application of the xallarap effect in the mass determination of microlenses, and the same method can be used to identify isolated dark lenses. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.12911-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.12911) | **SDSS J222428.53+261423.2: unique emission lines properties unveil a  sub-kiloparsec dual AGN candidate**  |
|| Q. Zheng, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-04-22*|
|*Comments*| *9 pages, 5 figures, accepted to be published in MNRAS Letter*|
|**Abstract**| In this paper, we presented a detailed analysis of the Sloan Digital Sky Survey optical spectrum of a new sub-kpc scale dual AGN candidate SDSS J222428.53+261423.2 (=SDSS J2224). The target is one of the few AGNs with all the optical narrow emission lines characterized by double-peaked profiles and with peak separations in velocity units of about 930 km/s. If the double-peaked narrow emission lines (DPNELs) are due to a dual AGN in \obj, the estimated physical separation between the two cores is about 500 pc. Meanwhile, three alternative explanations are also discussed in this paper, however, we can not find solid evidence to completely rule them out. Our results support the presence of a sub-kpc dual AGN with DPNELs in all lines, indicating a key episode of galaxy merging evolution at sub-kpc scale. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.13047-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.13047) | **Formation of a $33\,M_{\odot}$ black hole in a low-metallicity binary**  |
|| <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-04-22*|
|*Comments*| *7 pages, 4 figures, submitted to OJAp. Comments welcome*|
|**Abstract**| A $33\,M_\odot$ black hole (BH) was recently discovered in an 11.6-year binary only 590 pc from the Sun. The system, Gaia BH3, contains a $0.8\,M_\odot$ low-metallicity giant ($\rm [M/H]=-2.2$) and is kinematically part of the Galactic halo, suggesting that the BH formed from a low-metallicity massive star. I show that orbits similar to that of Gaia BH3 are naturally produced through isolated binary evolution. The system's period and eccentricity can result from a broad range of initial orbits with a modest natal kick ($v_{\rm kick}\lesssim 75\,\rm km\,s^{-1}$) to the BH. I construct MESA models for metal-poor massive stars with initial masses ranging from $35-55\,M_{\odot}$, which reach maximum radii of $1150-1800\,R_{\odot}$ as red supergiants. Stars of this size would fit inside most plausible pre-supernova orbits for the system without overflowing their Roche lobes. In addition, models with moderately rapid initial rotation ($\Omega/\Omega_{\rm crit} \gtrsim 0.45$) undergo chemically homogeneous evolution and never expand to radii larger than $10\,R_{\odot}$. There are thus multiple channels through which a low-metallicity, extreme-mass ratio binary could produce a system like Gaia BH3. Dynamical formation scenarios are also viable, and there is little doubt that both isolated and dynamically-formed BH binaries with orbits similar to Gaia BH3 will be discovered in Gaia DR4. Only about 1 in 10,000 stars in the solar neighborhood have metallicities as low as Gaia BH3. This suggests that BH companions are dramatically over-represented at low-metallicity, though caveats related to small number statistics apply. The fact that the luminous star in Gaia BH3 has been a giant - greatly boosting its detectability - only for $\sim$1% of the time since the system's formation implies that additional massive BHs remain to be discovered with only moderately fainter companions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2404.12432.md
    + _build/html/tmp_2404.12432/./SFH_binned_final.png
    + _build/html/tmp_2404.12432/./129149.png
    + _build/html/tmp_2404.12432/./128036.png
    + _build/html/tmp_2404.12432/./mass_tform.png
exported in  _build/html/2404.12443.md
    + _build/html/tmp_2404.12443/./CII-cont-combined-map.png
    + _build/html/tmp_2404.12443/./CII-tot-out-comp-spectra.png
    + _build/html/tmp_2404.12443/./J0923+0402moment-maps-vertical.png
    + _build/html/tmp_2404.12443/./onepc_t25_3ad6_c17_best_fit_model_1_2.png
    + _build/html/tmp_2404.12443/./onepc_t25_3ad6_c17_pars_2_1.png
    + _build/html/tmp_2404.12443/./mout-eout-pout-correlation.png
exported in  _build/html/2404.12585.md
    + _build/html/tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_uncalibrated_ATLASJ025-33.png
    + _build/html/tmp_2404.12585/./Plots/Predictions_FluxRatio_Sample1_Greig.png
    + _build/html/tmp_2404.12585/./Plots/Predictions_FluxRatio_Sample2_Greig.png
    + _build/html/t

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\white}{\textcolor{white}}$</div>



<div id="title">

# The JWST-SUSPENSE Ultradeep Spectroscopic Program: \ Survey Overview and Star-Formation Histories of Quiescent Galaxies at $\boldsymbol{1<z<3}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.12432-b31b1b.svg)](https://arxiv.org/abs/2404.12432)<mark>Appeared on: 2024-04-22</mark> -  _Submitted to ApJ; 24 pages, 13 figures, 2 tables (excluding appendices)_

</div>
<div id="authors">

M. Slob, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** We present an overview and first results from the Spectroscopic Ultradeep Survey Probing Extragalactic Near-infrared Stellar Emission (SUSPENSE), executed with NIRSpec on JWST. The primary goal of the SUSPENSE program is to characterize the stellar, chemical, and kinematic properties of massive quiescent galaxies at cosmic noon. In a single deep NIRSpec/MSA configuration, we target 20 distant quiescent galaxy candidates ( $z=1-3$ , $H_{AB}<23$ ), as well as 53 star-forming galaxies at $z=1-4$ . With 16 hr of integration and the G140M-F100LP dispersion-filter combination, we observe numerous Balmer and metal absorption lines for all quiescent candidates. We derive stellar masses (log $M_*/M_{\odot}\sim10.3-11.5$ ) and detailed star-formation histories (SFHs) and show that all 20 candidate quiescent galaxies indeed have quenched stellar populations. These galaxies show a variety of mass-weighted ages ( $0.8-3.0$ Gyr) and star formation timescales ( $\sim0.5-4$ Gyr), and four out of 20 galaxies were already quiescent by $z=3$ . On average, the $z>1.75$ $[z<1.75]$ galaxies formed 50 \% of their stellar mass before $z=4$ $[z=3]$ . Furthermore, the typical SFHs of galaxies in these two redshift bins ( $z_{\text{mean}}=2.2$ and $z_{\text{mean}}=1.3$ ) indicate that galaxies at higher redshift formed earlier and over shorter star-formation timescales compared to lower redshifts. Although this evolution is naturally explained by the growth of the quiescent galaxy population over cosmic time, we cannot rule out that mergers and late-time star formation also contribute to the evolution. In future work, we will further unravel the early formation, quenching, and late-time evolution of these galaxies by extending this work with studies on their chemical abundances, resolved stellar populations and kinematics.

</div>

<div id="div_fig1">

<img src="tmp_2404.12432/./SFH_binned_final.png" alt="Fig6" width="100%"/>

**Figure 6. -** Top panel: Distribution of the observed redshifts, separated in two redshift bins. The dashed lines indicate the mean redshift of each bin. Middle panel: the average SFH of all quiescent targets, separated into two redshift bins. Here the SFH corresponds to the total mass fraction formed as a function of age of the universe. Bottom panel: the summed SFR of all quiescent galaxies as a function of age of the universe, separated into two redshift bins. Before summing the SFRs of the galaxies we first normalize them by their total stellar mass. The shaded regions around the mean SFHs indicate the error on the mean. In the middle and bottom panels, the dotted vertical lines indicate the average $t_{\text{form}}$ of each redshift bin, with the vertical bands corresponding to the standard deviation of $t_{\text{form}}$. (*fig:SFH_binned*)

</div>
<div id="div_fig2">

<img src="tmp_2404.12432/./129149.png" alt="Fig11.1" width="50%"/><img src="tmp_2404.12432/./128036.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Overview of UltraVISTA photometric SEDs (bottom-left), NIRSpec spectra (right), and HST image (top-left) of two example quiescent galaxies in our survey. The 2D spectra for the two observed dithers and the combined 1D spectrum are shown in the top-right and bottom-right rows, respectively. Flux densities ($F_{\lambda}$) are in $10^{-19}$ erg s$^{-1}$ cm$^{-2}$ Å$^{-1}$. The coverage of the NIRSpec spectra are indicated in the SED panels by the gray rectangles. The best-fit \texttt{Prospector} models to the photometry and spectra are shown in red (see Section \ref{sec:prosp}). The images in the top-left panels show the HST-F160W imaging, with an overlay of the NIRSpec MSA slit orientation for one nod position. In Appendix \ref{ap:all_spec} we show this figure for all targeted quiescent galaxies. (*fig:example_q_spec*)

</div>
<div id="div_fig3">

<img src="tmp_2404.12432/./mass_tform.png" alt="Fig5" width="100%"/>

**Figure 5. -** Age of the universe where 50\% of the stellar mass in the galaxy has formed ($t_{\text{form}}$) as a function of stellar mass for all quiescent galaxies. The symbols are colored by the redshift at observation. We show the [Nanayakkara, Glazebrook and Jacobs (2024)]()$z\sim3-4$ sample as gray diamonds, and the [Beverage, Kriek and Suess (2023)]()(gray plus), [Glazebrook, Nanayakkara and Schreiber (2023)]()(gray square), [Setton, Khullar and Miller (2024)]()(gray cross), and [Kriek and Conroy (2016)]()(gray star). (*fig:mass_tform*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.12432"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\lcii}{L_{\rm[CII]}}$
$\newcommand{\lir}{L_{\rm IR}}$
$\newcommand{\zn}{J0923+0402}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\lbol}{L_{\rm Bol}}$</div>



<div id="title">

# Multi-phase black-hole feedback and a bright [CII] halo in a Lo-BAL quasar at $\mathbf{z\sim6.6}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.12443-b31b1b.svg)](https://arxiv.org/abs/2404.12443)<mark>Appeared on: 2024-04-22</mark> -  _Submitted to ApJ_

</div>
<div id="authors">

M. Bischetti, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Although the mass growth of supermassive black holes during the Epoch of Reionisation is expected to play a role in shaping the concurrent growth of their host-galaxies, observational evidence of feedback at z $\gtrsim$ 6 is still sparse. We perform the first multi-scale and multi-phase characterisation of black-hole driven outflows in the $z\sim6.6$ quasar J0923+0402 and assess how these winds impact the cold gas reservoir. We employ the SimBAL spectral synthesis to fit broad absorption line (BAL) features and find a powerful ionized outflow on $\lesssim210$ pc scale, with a kinetic power $\sim2-100$ \% of the quasar luminosity. ALMA observations of [ CII ] emission allow us to study the morphology and kinematics of the cold gas. We detect high-velocity [ CII ] emission, likely associated with a cold neutral outflow at $\sim0.5-2$ kpc scale in the host-galaxy, and a bright extended [ CII ] halo with a size of $\sim15$ kpc. For the first time at such an early epoch, we accurately constrain the outflow energetics in both the ionized and the atomic neutral gas phases. We find such energetics to be consistent with expectations for an efficient feedback mechanism, and both ejective and preventative feedback modes are likely at play. The scales and energetics of the ionized and atomic outflows suggest that they might be associated with different quasar accretion episodes. The results of this work indicate that strong black hole feedback is occurring in quasars at $z\gtrsim6$ and is likely responsible for shaping the properties of the cold gas reservoir up to circum-galactic scales.

</div>

<div id="div_fig1">

<img src="tmp_2404.12443/./CII-cont-combined-map.png" alt="Fig5.1" width="33%"/><img src="tmp_2404.12443/./CII-tot-out-comp-spectra.png" alt="Fig5.2" width="33%"/><img src="tmp_2404.12443/./J0923+0402moment-maps-vertical.png" alt="Fig5.3" width="33%"/>

**Figure 5. -** _ Left:_ Map of the [CII] emission in $\zn$. Solid contours correspond to [3,4,6,8,10,12,13]$\sigma$ significance, where $\sigma=0.017$ Jy beam$^{-1}$\kms. The same levels are shown for the continuum emission by the dashed contours, with $\sigma=5.9\times10^{-6}$ Jy beam$^{-1}$. The black cross indicates the quasar optical position, as measured from NIRCam F200W data, and the white plus sign corresponds to the peak of the 242-257 GHz continuum emission probed by ALMA. _Middle:_[CII] spectrum corresponding to the total emission detected at $\gtrsim3\sigma$(top panel). The best-fit Gaussian profile is shown by the red curve. Middle panel displays the [CII] spectrum extracted from the region with velocity dispersion higher than 230 $\kms$, including the quasar location. To model the line profile, we added a second blueshifted Gaussian component with $v\simeq-300$\kms to the scaled best-fit model of the top panel. Bottom panel shows the [CII] spectrum extracted from one beam aperture centred on the secondary continuum peak. _ Right:_ Velocity and velocity dispersion maps corresponding to the [CII] emission at $>3\sigma$. The black-dashed polygon highlights the triangle-shaped region with high-velocity dispersion (Sect. \ref{subsec:alma}). (*fig:cii-integrated*)

</div>
<div id="div_fig2">

<img src="tmp_2404.12443/./onepc_t25_3ad6_c17_best_fit_model_1_2.png" alt="Fig4.1" width="50%"/><img src="tmp_2404.12443/./onepc_t25_3ad6_c17_pars_2_1.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** _Left:_ X-Shooter spectrum of $\zn$(black) and best-fitting _SimBAL_ model (red).
    The dashed blue curve represents the unabsorbed quasar emission model by \citep{Bischetti23}.
    The noise spectrum and the spectral regions excluded from the fit are plotted in grey.
    Labels indicate the major UV emission lines. _Right:_ BAL outflow physical parameters as a function of velocity with error bars representing 95\% confidence regions.
    The top two panels show the fit parameters ($\log U$, $\log N_H-\log U$, $\log a$) used in _SimBAL_ modeling.
    The bottom panel shows the distribution of hydrogen column density ($\log N_H$) corrected for the covering fraction (Appendix \ref{App:simbal}).
     (*fig:simbal_bestfit*)

</div>
<div id="div_fig3">

<img src="tmp_2404.12443/./mout-eout-pout-correlation.png" alt="Fig6" width="100%"/>

**Figure 6. -** _ Left:_ Mass outflow rate as a function of bolometric luminosity for the [CII] and BAL outflow detected in $\zn$(stars). The shaded bars show the range of possible values according to our analysis. We compare our results with [CII] outflows from \cite{Bischetti19, Izumi21a, Izumi21, Tripodi22} and BAL outflows from \cite{Choi20,Choi22}(diamonds) and \cite{Miller20}(squares). We also include the compilation of black-hole driven molecular outflows from \cite{Bischetti19pds} and references therein, and recent measurements by \cite{RamosAlmeida22, Zanchettin21, Zanchettin23}.
    The estimates for the comparison BAL outflows were also calculated assuming a thin-shell scenario.
    The bolometric luminosity for the BAL outflow of $\zn$ is arbitrarily shifted for plotting. _ Middle:_ Outflow kinetic power as a function of $L_{\rm Bol}$. The solid, dashed and dotted lines correspond to $L_{\rm Bol}$, 0.1$L_{\rm Bol}$, and 0.01$L_{\rm Bol}$, respectively. _ Right:_ Outflow momentum flux as a function of the outflow velocity. The solid(dashed) line indicates the expected momentum boost for a black-hole driven wind arising with nuclear velocity 0.1$c$ and propagating with a momentum(energy) conserving expansion \citep{Zubovas&King12}.  (*fig:correlations*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.12443"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\lya}{\ifmmode\mathrm{Ly}\alpha\else Ly\alpha\fi}$
$\newcommand{\lyb}{\ifmmode\mathrm{Ly}\beta\else Ly\beta\fi}$
$\newcommand{\igm}{\ifmmode\mathrm{IGM}\else IGM\fi}$
$\newcommand{\lae}{\ifmmode\mathrm{LAE}\else LAE\fi}$
$\newcommand{\h}{\ifmmode\mathrm{H}\else H\fi}$
$\newcommand{\hi}{\ifmmode\mathrm{H {\scriptscriptstyle I}}\else H {\scriptsize I}\fi}$
$\newcommand{\hii}{\ifmmode\mathrm{H {\scriptscriptstyle II}}\else H {\scriptsize II}\fi}$
$\newcommand{\cmb}{\ifmmode\mathrm{CMB}\else CMB\fi}$
$\newcommand{\qso}{\ifmmode\mathrm{QSO}\else QSO\fi}$
$\newcommand{\eor}{\ifmmode\mathrm{EoR}\else EoR\fi}$
$\newcommand{\heii}{\ifmmode\mathrm{He {\scriptscriptstyle II}}\else He {\scriptsize II}\fi}$
$\newcommand{\heiii}{\ifmmode\mathrm{He {\scriptscriptstyle III}}\else He {\scriptsize III}\fi}$
$\newcommand{\ciii}{\ifmmode\mathrm{C {\scriptscriptstyle III]}}\else C {\scriptsize III]}\fi}$
$\newcommand{\oiii}{\ifmmode\mathrm{O {\scriptscriptstyle III}}\else O {\scriptsize III}\fi}$
$\newcommand{\aliii}{\ifmmode\mathrm{Al {\scriptscriptstyle III}}\else Al {\scriptsize III}\fi}$
$\newcommand{\mgii}{\ifmmode\mathrm{Mg {\scriptscriptstyle II}}\else Mg {\scriptsize II}\fi}$
$\newcommand{\fe}{\ifmmode\mathrm{Fe}\else Fe\fi}$
$\newcommand{\nv}{\ifmmode\mathrm{N {\scriptscriptstyle V}}\else N {\scriptsize V}\fi}$
$\newcommand{\niv}{\ifmmode\mathrm{N {\scriptscriptstyle IV]}}\else N {\scriptsize IV]}\fi}$
$\newcommand{\cii}{\ifmmode\mathrm{C {\scriptscriptstyle II}}\else C {\scriptsize II}\fi}$
$\newcommand{\civ}{\ifmmode\mathrm{C {\scriptscriptstyle IV}}\else C {\scriptsize IV}\fi}$
$\newcommand{\siv}{\ifmmode\mathrm{Si {\scriptscriptstyle IV}}\else Si {\scriptsize IV}\fi}$
$\newcommand{\siii}{\ifmmode\mathrm{Si {\scriptscriptstyle II}}\else Si {\scriptsize II}\fi}$
$\newcommand{\siiii}{\ifmmode\mathrm{Si {\scriptscriptstyle III]}}\else Si {\scriptsize III]}\fi}$
$\newcommand{\ovi}{\ifmmode\mathrm{O {\scriptscriptstyle VI}}\else O {\scriptsize VI}\fi}$
$\newcommand{\sioiv}{\ifmmode\mathrm{Si {\scriptscriptstyle IV} \plus O {\scriptscriptstyle IV]}}\else Si {\scriptsize IV} +O {\scriptsize IV]}\fi}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\note}[1]{{\color{red} #1}}$
$\newcommand{\comment}[1]{{\color{blue} #1}}$
$\newcommand{\nf}{x_{\rm HI}}$
$\newcommand{\avenf}{\bar{x}_{\rm HI}}$
$\newcommand{\faintgal}{\textsc{\small Faint galaxies}}$
$\newcommand{\brightgal}{\textsc{\small Bright galaxies}}$
$\newcommand{\smallHII}{\textsc{\small Small \hii }}$
$\newcommand{\intermediateHII}{\textsc{\small Intermediate \hii }}$
$\newcommand{\largeHII}{\textsc{\small Large \hii }}$
$\newcommand{\cmmc}{\textsc{\small 21CMMC}}$
$\newcommand{\cmfst}{\textsc{\small 21CMFAST}}$
$\newcommand{\sense}{{\small 21}\textsc{cmsense}}$
$\newcommand{\CH}{\textsc{cosmohammer}}$
$\newcommand{\emcee}{\textsc{emcee}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# IGM damping wing constraints on the tail end of reionisation from the enlarged XQR-30 sample

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.12585-b31b1b.svg)](https://arxiv.org/abs/2404.12585)<mark>Appeared on: 2024-04-22</mark> -  _21 pages, 9 figures and 2 tables. Accepted for publication in MNRAS_

</div>
<div id="authors">

B. Greig, et al. -- incl., <mark>E. Bañados</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ The attenuation of Ly $\alpha$ photons by neutral hydrogen in the intergalactic medium (IGM) at $z\gtrsim5$ continues to be a powerful probe for studying the epoch of reionisation. Given a framework to estimate the intrinsic (true) Ly $\alpha$ emission of high- $z$ sources, one can infer the ionisation state of the IGM during reionisation. In this work, we use the enlarged XQR-30 sample of 42 high-resolution and high-SNR QSO spectra between $5.8\lesssim z\lesssim 6.6$ obtained with VLT/X-Shooter to place constraints on the IGM neutral fraction. This is achieved using our existing Bayesian QSO reconstruction framework which accounts for uncertainties such as the: (i) posterior distribution of predicted intrinsic $\lya$ emission profiles (obtained via covariance matrix reconstruction of the Ly $\alpha$ and $\nv$ emission lines from unattenuated high-ionisation emission line profiles; $\civ$ , $\sioiv$ and $\ciii$ ) and (ii) distribution of ionised regions within the IGM using synthetic damping wing profiles drawn from a $1.6^3$ Gpc $^3$ reionisation simulation. Following careful quality control, we used 23 of the 42 available QSOs to obtain constraints/limits on the IGM neutral fraction during the tail-end of reionisation. Our median and 68th percentile constraints on the IGM neutral fraction are: $0.20\substack{+0.14\\-0.12}$ and $0.29\substack{+0.14\\-0.13}$ at $z = 6.15$ and 6.35. Further, we also report 68th percentile upper-limits of $\bar{x}_{\hi } < 0.21$ , 0.20, 0.21 and 0.18 at $z = 5.8, 5.95, 6.05$ and 6.55. These results imply reionisation is still ongoing at $5.8\lesssim z\lesssim 6.55$ , consistent with previous results from XQR-30 (dark fraction and Ly $\alpha$ forest) along with other observational probes considered in the literature.

</div>

<div id="div_fig1">

<img src="tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_uncalibrated_ATLASJ025-33.png" alt="Fig2" width="100%"/>

**Figure 2. -** A visual demonstration of the initial covariance reconstruction for ATLAS J025-33 at $z=6.34$ highlighting problems with our original reconstruction pipeline. _Left panel:_ The observed QSO spectrum (black curve), corresponding maximum-likelihood (red curve) reconstruction and 300 random profile draws (grey curves) from our full reconstructed covariance matrix. The solid purple curve corresponds to the median reconstruction profile obtained from the full distribution of reconstructed profiles while the dashed (dotted) curves correspond to the 68th (95th) distribution of profiles. The vertical blue dashed line corresponds to rest-frame $\lya$  while the purple box denotes the region over which we fit for the IGM damping wing imprint (1218-1230Å). _Right panel:_ The observed $\lya$  transmission profile (black curve) obtained by dividing the observed spectrum by the median reconstructed profile. The coloured lines correspond to the median synthetic IGM damping wing profiles for each IGM neutral fraction value sampled by our EoR simulations. Grey shaded regions correspond to absorption features that are masked by our damping wing fitting pipeline. (*fig:uncalibrated*)

</div>
<div id="div_fig2">

<img src="tmp_2404.12585/./Plots/Predictions_FluxRatio_Sample1_Greig.png" alt="Fig3.1" width="50%"/><img src="tmp_2404.12585/./Plots/Predictions_FluxRatio_Sample2_Greig.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** The performance of our covariance matrix reconstruction method from two distinct samples of 30 QSOs from X-shooter (left panel) and SDSS (BOSS; right panel) at $3.5 < z < 4.5$. The red solid lines correspond to the flux ratio (reconstructed maximum-likelihood profile over true intrinsic profile) for each individual QSO in the samples. The black solid lines correspond to the median determined over each sample. (*fig:Correction*)

</div>
<div id="div_fig3">

<img src="tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_inset_PSOJ029-29.png" alt="Fig5.1" width="12%"/><img src="tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_inset_ATLASJ029-36.png" alt="Fig5.2" width="12%"/><img src="tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_inset_VDESJ0224-47.png" alt="Fig5.3" width="12%"/><img src="tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_inset_PSOJ060+24.png" alt="Fig5.4" width="12%"/><img src="tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_inset_J0408-5632.png" alt="Fig5.5" width="12%"/><img src="tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_inset_PSOJ108+08.png" alt="Fig5.6" width="12%"/><img src="tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_inset_SDSSJ0842+1218.png" alt="Fig5.7" width="12%"/><img src="tmp_2404.12585/./Plots/QSO_ReconProfilesIndividual_inset_PSOJ158-14.png" alt="Fig5.8" width="12%"/>

**Figure 5. -** Intrinsic QSO reconstructions of the XQR-30 sample. We provide the maximum-likelihood reconstruction (red curve), the purple solid, dashed and dotted curves correspond to the median, 68th and 95th percentile profiles and the thin grey curves represent 300 random draws from our full posterior distribution. The solid thick grey curve near zero corresponds to the QSO noise spectrum, amplified by a factor of 5 to improve the numerical performance of our neutral fraction inference approach (see text for further details). The purple shaded box demarcates our damping wing fitting region (1218-1230Å region). _Inset panel:_ The $\lya$  transmission profile highlighting the presence (if any) of an IGM damping wing imprint (i.e. observed transmission spectrum below unity) assuming the median (purple curve). Grey shaded regions correspond to features in the observed spectrum removed from our likelihood fitting. (*fig:results*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.12585"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\wei}{\color{red}}$
$\newcommand{\remove}{\color{lightgray}}$
$\newcommand{\zhecheng}{\color{cyan}}$
$\newcommand{\eventname}{OGLE-2015-BLG-0845 }$
$\newcommand{\spitzer}{\emph{Spitzer} }$
$\newcommand{\anupiE}{\pi_{\rm{E}} = 2.60^{+0.31}_{-0.22}}$
$\newcommand{\satpiEs}{\pi_{\rm{E}} = 0.087^{+0.002}_{-0.002}}$
$\newcommand{\satpiEl}{\pi_{\rm{E}} = 0.127^{+0.003}_{-0.002}}$
$\newcommand{\DeltaChisrPX}{\Delta \chi^{2} \simeq  340}$
$\newcommand{\DeltaChisrTwoSPX}{\Delta \chi^{2} \simeq  50}$
$\newcommand{\binsDeltachisq}{220}$
$\newcommand{\binlDeltachisq}{180}$
$\newcommand{\SonePhot}{(V-I, I)_{\rm{S1}} = (1.46 \pm 0.04, 18.65 \pm 0.13)}$
$\newcommand{\StwoPhot}{(V-I, I)_{\rm{S2}} = (1.4\pm 0.3, 20.6\pm 0.5)}$
$\newcommand{\RCzero}{(V-I, I)_{\rm{RC,0}} = (1.06 \pm 0.06, 14.46 \pm 0.04)}$
$\newcommand{\RCfit}{(V-I, I)_{\rm{RC}} = (1.972\pm0.012, 15.82\pm 0.07)}$
$\newcommand{\Extinction}{(E_{\rm{S}}(V-I), A_{\rm{S}}(I)) = (0.91 \pm 0.06, 1.37 \pm 0.08)}$
$\newcommand{\SonePhotZero}{(V-I, M_{\rm{I}} )_{\rm{S1,0}} = (0.54 \pm 0.07, 2.7 \pm 0.3)}$
$\newcommand{\StwoPhotZero}{(V-I, M_{\rm{I}} )_{\rm{S2,0}} = (0.5 \pm 0.4, 4.7 \pm 0.5)}$
$\newcommand{\orcid}[1]{\href{https://orcid.org/#1}{\textcolor[HTML]{A6CE39}{\aiOrcid}}}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# OGLE-2015-BLG-0845L: A low-mass M dwarf from the microlensing parallax and xallarap effects

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.13031-b31b1b.svg)](https://arxiv.org/abs/2404.13031)<mark>Appeared on: 2024-04-22</mark> -  _13 pages, 10 figures. Submitted to Monthly Notices of the Royal Astronomical Society_

</div>
<div id="authors">

Z. Hu, et al. -- incl., <mark>A. Gould</mark>

</div>
<div id="abstract">

**Abstract:** We present the analysis of the microlensing event OGLE-2015-BLG-0845, which was affected by both the microlensing parallax and xallarap effects. The former was detected via the simultaneous observations from the ground and Spitzer, and the latter was caused by the orbital motion of the source star in a relatively close binary. The combination of these two effects led to a direct mass measurement of the lens object, revealing a low-mass ( $0.14 \pm 0.05 M_{\sun}$ ) M-dwarf at the bulge distance ( $7.6 \pm 1.0$ kpc). The source binary consists of a late F-type subgiant and a K-type dwarf of $\sim1.2 M_{\sun}$ and $\sim 0.9 M_{\sun}$ , respectively, and the orbital period is $70 \pm 10$ days. OGLE-2015-BLG-0845 is the first single-lens event in which the lens mass is measured via the binarity of the source. Given the abundance of binary systems as potential microlensing sources, the xallarap effect may not be a rare phenomenon. Our work thus highlights the application of the xallarap effect in the mass determination of microlenses, and the same method can be used to identify isolated dark lenses.

</div>

<div id="div_fig1">

<img src="tmp_2404.13031/./fig/lc_satprlx_x1sp_x2sp.png" alt="Fig8" width="100%"/>

**Figure 8. -** 
OGLE $I$ and $V$ band light curves (left panels) and $\spitzer$ 3.6 $\mu$m light curve (right panels) of the microlensing event $\eventname$. In each panel, the best-fit 1L1S parallax model, 1L1S parallax + xallarap model, and the 1L2S parallax + xallarap model are shown as dashed gray line, dash-dotted line, and solid line, respectively. The OGLE $V$-band data have been aligned to the OGLE $I$-band data based on the best-fit flux parameters, and the $\spitzer$ flux values are shown in logarithmic scale (quasi-magnitudes).
The top panels show the observed data and the best-fit models, and the middle panels show the residuals of data and the other models relative to the 1L1S parallax model. In the lower panels, we show the cumulative $\Delta \chi^2$ distribution of the other two models relative to the 1L1S parallax model.
 (*fig:lc-all*)

</div>
<div id="div_fig2">

<img src="tmp_2404.13031/./fig/q_and_isochrone.png" alt="Fig4" width="100%"/>

**Figure 4. -** 
    Comparison between the binary mass from the isochrone fitting and the mass ratio $q$ from light curve fitting.
    The red line represents the $q=1$ line.
    The stellar masses determined by isochrone fitting are shown in black contours, with the solid and dashed contours showing the 1 and 2 sigma regions.
    The light blue shadow regions mark the mass ratio 90\% upper limit of each degenerate model.
 (*fig:q-iso*)

</div>
<div id="div_fig3">

<img src="tmp_2404.13031/./fig/microlensing-geo-en-xlrp+prlx.png" alt="Fig7" width="100%"/>

**Figure 7. -** An illustration of the satellite parallax method and the xallarap effect in the microlensing phenomenon. Distances and physical sizes are not to scale. The microlensing source is a binary system, with the yellow circle indicating the primary star (i.e., primary source) and the brown circle indicating the secondary star (i.e., secondary source). The centre of light, namely the photocentre, of the source system is indicated by the orange circle. On the observer plane, Spitzer was in an Earth-trailing orbit and observed the same source simultaneously as the ground-based observatories.
     (*fig:xlrp_geo*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.13031"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

409  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
